# 层和块
:label:`sec_model_construction`

之前首次介绍神经网络时，我们关注的是具有单一输出的线性模型。
在这里，整个模型只有一个输出。
注意，单个神经网络
（1）接受一些输入；
（2）生成相应的标量输出；
（3）具有一组相关 *参数*（parameters），更新这些参数可以优化某目标函数。

然后，当考虑具有多个输出的网络时，
我们利用矢量化算法来描述整层神经元。
像单个神经元一样，层（1）接受一组输入，
（2）生成相应的输出，
（3）由一组可调整参数描述。
当我们使用softmax回归时，一个单层本身就是模型。
然而，即使我们随后引入了多层感知机，我们仍然可以认为该模型保留了上面所说的基本架构。

对于多层感知机而言，整个模型及其组成层都是这种架构。
整个模型接受原始输入（特征），生成输出（预测），
并包含一些参数（所有组成层的参数集合）。
同样，每个单独的层接收输入（由前一层提供），
生成输出（到下一层的输入），并且具有一组可调参数，
这些参数根据从下一层反向传播的信号进行更新。

事实证明，研究讨论“比单个层大”但“比整个模型小”的组件更有价值。
例如，在计算机视觉中广泛流行的ResNet-152架构就有数百层，
这些层是由*层组*（groups of layers）的重复模式组成。
这个ResNet架构赢得了2015年ImageNet和COCO计算机视觉比赛
的识别和检测任务 :cite:`He.Zhang.Ren.ea.2016`。
目前ResNet架构仍然是许多视觉任务的首选架构。
在其他的领域，如自然语言处理和语音，
层组以各种重复模式排列的类似架构现在也是普遍存在。

为了实现这些复杂的网络，我们引入了神经网络*块*的概念。
*块*（block）可以描述单个层、由多个层组成的组件或整个模型本身。
使用块进行抽象的一个好处是可以将一些块组合成更大的组件，
这一过程通常是递归的，如 :numref:`fig_blocks`所示。
通过定义代码来按需生成任意复杂度的块，
我们可以通过简洁的代码实现复杂的神经网络。

![多个层被组合成块，形成更大的模型](http://d2l.ai/_images/blocks.svg)
:label:`fig_blocks`

从编程的角度来看，块由*类*（class）表示。
它的任何子类都必须定义一个将其输入转换为输出的前向传播函数，
并且必须存储任何必需的参数。
注意，有些块不需要任何参数。
最后，为了计算梯度，块必须具有反向传播函数。
在定义我们自己的块时，由于自动微分（在 :numref:`sec_autograd` 中引入）
提供了一些后端实现，我们只需要考虑前向传播函数和必需的参数。

在构造自定义块之前，(**我们先回顾一下多层感知机**)
（ :numref:`sec_mlp_concise` ）的代码。
下面的代码生成一个网络，其中包含一个具有256个单元和ReLU激活函数的全连接隐藏层，
然后是一个具有10个隐藏单元且不带激活函数的全连接输出层。


In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0343,  0.0264,  0.2505, -0.0243,  0.0945,  0.0012, -0.0141,  0.0666,
         -0.0547, -0.0667],
        [ 0.0772, -0.0274,  0.2638, -0.0191,  0.0394, -0.0324,  0.0102,  0.0707,
         -0.1481, -0.1031]], grad_fn=<AddmmBackward0>)

在这个例子中，我们通过实例化`nn.Sequential`来构建我们的模型，
层的执行顺序是作为参数传递的。
简而言之，(**`nn.Sequential`定义了一种特殊的`Module`**)，
即在PyTorch中表示一个块的类，
它维护了一个由`Module`组成的有序列表。
注意，两个全连接层都是`Linear`类的实例，
`Linear`类本身就是`Module`的子类。
另外，到目前为止，我们一直在通过`net(X)`调用我们的模型来获得模型的输出。
这实际上是`net.__call__(X)`的简写。
这个前向传播函数非常简单：
它将列表中的每个块连接在一起，将每个块的输出作为下一个块的输入。


## [**自定义块**]

要想直观地了解块是如何工作的，最简单的方法就是自己实现一个。
在实现我们自定义块之前，我们简要总结一下每个块必须提供的基本功能。


1. 将输入数据作为其前向传播函数的参数。
1. 通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。
1. 计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。
1. 存储和访问前向传播计算所需的参数。
1. 根据需要初始化模型参数。


在下面的代码片段中，我们从零开始编写一个块。
它包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层。
注意，下面的`MLP`类继承了表示块的类。
我们的实现只需要提供我们自己的构造函数（Python中的`__init__`函数）和前向传播函数。


In [2]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

我们首先看一下前向传播函数，它以`X`作为输入，
计算带有激活函数的隐藏表示，并输出其未规范化的输出值。
在这个`MLP`实现中，两个层都是实例变量。
要了解这为什么是合理的，可以想象实例化两个多层感知机（`net1`和`net2`），
并根据不同的数据对它们进行训练。
当然，我们希望它们学到两种不同的模型。

接着我们[**实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层**]。
注意一些关键细节：
首先，我们定制的`__init__`函数通过`super().__init__()`
调用父类的`__init__`函数，
省去了重复编写模版代码的痛苦。
然后，我们实例化两个全连接层，
分别为`self.hidden`和`self.out`。
注意，除非我们实现一个新的运算符，
否则我们不必担心反向传播函数或参数初始化，
系统将自动生成这些。

我们来试一下这个函数：


In [3]:
net = MLP()
net(X)

tensor([[ 0.0669,  0.2202, -0.0912, -0.0064,  0.1474, -0.0577, -0.3006,  0.1256,
         -0.0280,  0.4040],
        [ 0.0545,  0.2591, -0.0297,  0.1141,  0.1887,  0.0094, -0.2686,  0.0732,
         -0.0135,  0.3865]], grad_fn=<AddmmBackward0>)

块的一个主要优点是它的多功能性。
我们可以子类化块以创建层（如全连接层的类）、
整个模型（如上面的`MLP`类）或具有中等复杂度的各种组件。
我们在接下来的章节中充分利用了这种多功能性，
比如在处理卷积神经网络时。

## [**顺序块**]

现在我们可以更仔细地看看`Sequential`类是如何工作的，
回想一下`Sequential`的设计是为了把其他模块串起来。
为了构建我们自己的简化的`MySequential`，
我们只需要定义两个关键函数：

1. 一种将块逐个追加到列表中的函数；
1. 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

下面的`MySequential`类提供了与默认`Sequential`类相同的功能。


In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

`__init__`函数将每个模块逐个添加到有序字典`_modules`中。
读者可能会好奇为什么每个`Module`都有一个`_modules`属性？
以及为什么我们使用它而不是自己定义一个Python列表？
简而言之，`_modules`的主要优点是：
在模块的参数初始化过程中，
系统知道在`_modules`字典中查找需要初始化参数的子块。


当`MySequential`的前向传播函数被调用时，
每个添加的块都按照它们被添加的顺序执行。
现在可以使用我们的`MySequential`类重新实现多层感知机。


In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 2.2759e-01, -4.7003e-02,  4.2846e-01, -1.2546e-01,  1.5296e-01,
          1.8972e-01,  9.7048e-02,  4.5479e-04, -3.7986e-02,  6.4842e-02],
        [ 2.7825e-01, -9.7517e-02,  4.8541e-01, -2.4519e-01, -8.4580e-02,
          2.8538e-01,  3.6861e-02,  2.9411e-02, -1.0612e-01,  1.2620e-01]],
       grad_fn=<AddmmBackward0>)

请注意，`MySequential`的用法与之前为`Sequential`类编写的代码相同
（如 :numref:`sec_mlp_concise` 中所述）。

## [**在前向传播函数中执行代码**]

`Sequential`类使模型构造变得简单，
允许我们组合新的架构，而不必定义自己的类。
然而，并不是所有的架构都是简单的顺序架构。
当需要更强的灵活性时，我们需要定义自己的块。
例如，我们可能希望在前向传播函数中执行Python的控制流。
此外，我们可能希望执行任意的数学运算，
而不是简单地依赖预定义的神经网络层。

到目前为止，
我们网络中的所有操作都对网络的激活值及网络的参数起作用。
然而，有时我们可能希望合并既不是上一层的结果也不是可更新参数的项，
我们称之为*常数参数*（constant parameter）。
例如，我们需要一个计算函数
$f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$的层，
其中$\mathbf{x}$是输入，
$\mathbf{w}$是参数，
$c$是某个在优化过程中没有更新的指定常量。
因此我们实现了一个`FixedHiddenMLP`类，如下所示：


In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

在这个`FixedHiddenMLP`模型中，我们实现了一个隐藏层，
其权重（`self.rand_weight`）在实例化时被随机初始化，之后为常量。
这个权重不是一个模型参数，因此它永远不会被反向传播更新。
然后，神经网络将这个固定层的输出通过一个全连接层。

注意，在返回输出之前，模型做了一些不寻常的事情：
它运行了一个while循环，在$L_1$范数大于$1$的条件下，
将输出向量除以$2$，直到它满足条件为止。
最后，模型返回了`X`中所有项的和。
注意，此操作可能不会常用于在任何实际任务中，
我们只展示如何将任意代码集成到神经网络计算的流程中。


In [7]:
net = FixedHiddenMLP()
net(X)

tensor(0.1862, grad_fn=<SumBackward0>)

我们可以[**混合搭配各种组合块的方法**]。
在下面的例子中，我们以一些想到的方法嵌套块。


In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.2183, grad_fn=<SumBackward0>)

## 效率


读者可能会开始担心操作效率的问题。
毕竟，我们在一个高性能的深度学习库中进行了大量的字典查找、
代码执行和许多其他的Python代码。
Python的问题[全局解释器锁](https://wiki.python.org/moin/GlobalInterpreterLock)
是众所周知的。
在深度学习环境中，我们担心速度极快的GPU可能要等到CPU运行Python代码后才能运行另一个作业。


## 小结

* 一个块可以由许多层组成；一个块可以由许多块组成。
* 块可以包含代码。
* 块负责大量的内部处理，包括参数初始化和反向传播。
* 层和块的顺序连接由`Sequential`块处理。

## 练习

1. 如果将`MySequential`中存储块的方式更改为Python列表，会出现什么样的问题？
1. 实现一个块，它以两个块为参数，例如`net1`和`net2`，并返回前向传播中两个网络的串联输出。这也被称为平行块。
1. 假设我们想要连接同一网络的多个实例。实现一个函数，该函数生成同一个块的多个实例，并在此基础上构建更大的网络。


In [5]:
import torch.nn as nn
import torch

# print一个时间戳
import datetime

print(datetime.datetime.now())  # 打印当前日期和时间


class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X


class MySequential_list(nn.Module):
    # 使用list
    def __init__(self, *args):
        # super(MySequential_list, self).__init__()
        super().__init__()
        self.sequential = []
        for module in args:
            self.sequential.append(module)

    def forward(self, X):
        for module in self.sequential:
            X = module(X)
        return X


X = torch.rand(1, 10)
net = MySequential(nn.Linear(10, 20), nn.ReLU(), nn.Linear(20, 10))
net_list = MySequential_list(nn.Linear(10, 20), nn.ReLU(), nn.Linear(20, 10))

# 分割线
print("使用_modules构建的网络：")
# 结果一样
print("net" + str(net(X)))
print("net_list" + str(net_list(X)))
# 使用_modules方便打印net的网络结构和参数，而list则无法做到
print("net", net, "\n", net.state_dict())
print("net_list", net_list, "\n", net_list.state_dict())

2025-07-02 23:42:43.749811
使用_modules构建的网络：
nettensor([[ 0.2175, -0.0927, -0.1648, -0.2101, -0.4045,  0.0931, -0.0378,  0.1357,
          0.3302, -0.0201]], grad_fn=<AddmmBackward0>)
net_listtensor([[ 0.3731, -0.2809,  0.4660, -0.0204,  0.0359,  0.1167, -0.2796, -0.0167,
         -0.2883,  0.0601]], grad_fn=<AddmmBackward0>)
net MySequential(
  (0): Linear(in_features=10, out_features=20, bias=True)
  (1): ReLU()
  (2): Linear(in_features=20, out_features=10, bias=True)
) 
 OrderedDict([('0.weight', tensor([[-3.7285e-02,  1.5642e-01,  8.3753e-02,  9.0733e-02, -2.7807e-02,
          5.8402e-02,  2.7955e-01, -2.2686e-01, -1.1135e-01,  1.9354e-01],
        [-1.2339e-01,  1.8890e-01,  7.0568e-02,  2.2030e-01,  1.8596e-01,
         -1.2804e-01, -2.4369e-02, -2.4155e-01, -1.4666e-01, -1.7872e-01],
        [-1.6214e-01, -2.4237e-01, -2.9907e-01,  1.6388e-01, -3.7799e-02,
         -2.5017e-01, -2.8916e-02, -1.0010e-01, -1.0258e-01, -2.2336e-01],
        [ 2.7936e-01, -6.4795e-02,  1.1188e-01, 

In [6]:
import torch.nn as nn
import torch


class Parallel(nn.Module):
    def __init__(self, net1, net2):
        super().__init__()
        self.net1 = net1  # 第一个子网络
        self.net2 = net2  # 第二个子网络

    def forward(self, X):
        x1 = self.net1(X)  # 第一个子网络的输出
        x2 = self.net2(X)  # 第二个子网络的输出
        return torch.cat((x1, x2), dim=1)  # 在通道维度上合并输出结果


X = torch.rand(2, 10)  # 输入数据
net = Parallel(
    nn.Sequential(nn.Linear(10, 12), nn.ReLU()),
    nn.Sequential(nn.Linear(10, 24), nn.ReLU()),
)  # 实例化并行网络
output = net(X)
output.size()  # 输出结果的大小

torch.Size([2, 36])

In [9]:
import torch.nn as nn
import torch


def create_network(num_instances, input_size, hidden_size, output_size):
    # # 创建一个线性层
    # linear_layer = nn.Sequential(
    #     nn.Linear(input_size, hidden_size),
    #     nn.ReLU(),
    #     nn.Linear(hidden_size, input_size),
    # )

    # # 创建多个实例并连接
    # instances = [linear_layer for _ in range(num_instances)]

    # 创建多个实例并连接，每次都新建一个nn.Sequential对象
    instances = [
        nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size),
        )
        for _ in range(num_instances)
    ]

    network = nn.Sequential(*instances)

    # 添加输出层
    output_layer = nn.Linear(input_size, output_size)
    network.add_module("output", output_layer)

    return network


# 示例用法
net = create_network(num_instances=3, input_size=10, hidden_size=5, output_size=2)
net.state_dict()  # 打印网络参数字典

OrderedDict([('0.0.weight',
              tensor([[-0.2500,  0.2120, -0.0722, -0.2493,  0.2813,  0.2781,  0.2507, -0.0161,
                       -0.2431,  0.1445],
                      [ 0.1241, -0.0080,  0.0309, -0.2603, -0.1185,  0.0293, -0.2308, -0.0344,
                        0.2725,  0.0583],
                      [ 0.2467, -0.1879, -0.0973,  0.1998, -0.1370,  0.2092,  0.2551, -0.1347,
                        0.1264,  0.0628],
                      [ 0.2235,  0.1986, -0.2631,  0.2237, -0.1921, -0.0627, -0.2036,  0.2576,
                       -0.0194, -0.2437],
                      [-0.0669, -0.2851, -0.0737, -0.2554, -0.1279, -0.3111, -0.0438,  0.1669,
                       -0.3143,  0.0372]])),
             ('0.0.bias',
              tensor([-0.0727,  0.1170, -0.0518, -0.2234,  0.1148])),
             ('0.2.weight',
              tensor([[-0.3332, -0.3731, -0.3879,  0.3411,  0.0995],
                      [-0.3597, -0.3364,  0.2723,  0.3077,  0.3312],
                      

[Discussions](https://discuss.d2l.ai/t/1827)
